In [4]:
import ee
import geemap

Map = geemap.Map()
import geemap

Map = geemap.Map()

trainData = ee.FeatureCollection("projects/ee-epgalanis/assets/Final_data_9k"),
Region = ee.Geometry.Polygon(
  [[[140.96126303942492,-33.99133319528273],
  [140.96126303942492,-38.110919172137265],
  [146.30061850817492,-39.02144634571554],
  [150.07991538317492,-37.52075013904677],
  [140.96126303942492,-33.99133319528273]]]),
Sentinel2A = ee.ImageCollection("COPERNICUS/S2_SR")



LAND_CLASS = 0
WATER_CLASS = 1
BANDS = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']

ic = Sentinel2A \
      .filterDate('2019-01-01', '2019-12-01') \
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))

Map.setCenter(143.96306, -37.74696, 12)

# # print (classifier.explain())

input = ic.median().select(BANDS)

split = 0.7;  # Roughly 70% training, 30% testing.
samples = trainData.randomColumn('random')
training = samples.filter(ee.Filter.lt('random', split))
validation = samples.filter(ee.Filter.gte('random', split))

# CART
# classifier = ee.Classifier.smileCart().train(training, 'class', BANDS)
#RandomForest
classifier = ee.Classifier.smileRandomForest(400).train(training, 'class', BANDS)

print('Training size:', training.size())
print('Test size:', validation.size())

classified = input.classify(classifier)
trainAccuracy = classifier.confusionMatrix()

print(' Training Confusion Matrix   ')
print('                 True         ')
print('                 Water   Land ')
print('Predicted  Water ' + trainAccuracy.getInfo()[0][0] + '      ' + trainAccuracy.getInfo()[0][1])
print('           Land  ' + trainAccuracy.getInfo()[1][0] + '      ' + trainAccuracy.getInfo()[1][1])

print('------------------------------')

# Classify the validation data.
validated = validation.classify(classifier)
# testAccuracy = validated.confusionMatrix()
testAccuracy = validated.errorMatrix('class', 'classification')

print('      Test Confusion Matrix   ')

print('                 True         ')
print('                 Water   Land ')
print('Predicted  Water ' + testAccuracy.getInfo()[0][0] + '      ' + testAccuracy.getInfo()[0][1])
print('           Land  ' + testAccuracy.getInfo()[1][0] + '      ' + testAccuracy.getInfo()[1][1])
print('------------------------------')

# Get a confusion matrix representing expected accuracy.
print('Validation overall accuracy: ', testAccuracy.accuracy())

# Here, we're using Region polygon as just a dummy feature
#dummy = ee.Feature(Region).set({'classifier': ee.String(classifier.explain().get('tree'))})
#Export.table.toAsset(ee.FeatureCollection(dummy), 'serialize-cart-classifier', 'cart_classifier_3')

#dummy = ee.Serializer.toJSON(classifier)

#Export.table.toAsset(ee.FeatureCollection(ee.Feature(Region).set({'classifier':dummy})),
#                    'serialize-randomforest-classifier',
#                    'randomforest_classifier_0')
trees = ee.List(ee.Dictionary(classifier.explain()).get('trees'))
dummy = ee.Feature(Region)

def func_pvg(x):
    return dummy.set('tree',x)})

col = ee.FeatureCollection(trees.map(function(x){return dummy.set('tree',x)}
col = ee.FeatureCollection(trees.map(func_pvg
))
Export.table.toAsset(col,'save_classifier','RandomForest01')

palette = [
  'A52A2A', # Land
  '0000FF', # Water
]

Map.addLayer(classified.clip(Region), {'palette': palette, 'min': 0, 'max': 2}, 'classification CART')
MapMap


SyntaxError: invalid syntax (789125576.py, line 87)